In [1]:
import numpy as np 
import nevis 

h = nevis.gb()

area = np.load('../res/area-with-sea.npy')
label = np.load('../res/label.npy')
maxima = np.load('../res/maxima.npy')
path_sum = np.load('../res/path_sum.npy')

expected_fe = path_sum / area

- `maxima` = [($x_0, y_0$), ... ], where $(x_i, y_i)$ is the grid coordinate of the $i$-th hill, sorted from the highest to the lowest. The length of maxima is the number of hills = number of basin of attraction.
- `label[x][y]` means grid coordinate $(x, y)$ belongs to the $i$-th basin of attraction (which belongs to the $i$-th hill).
- `area[i]` returns the area of the $i$-th basin of attraction (including sea).
- `path_sum[i]` returns the sum of (gradient ascending) path lengths from each point to the $i$-th local max in the $i$-th b.o.a. 
- `path_sum[i] / area[i]` is the average path length (function evaluation) from each point in the $i$-th b.o.a to the $i$-th local max.

In [2]:
# check that the sum of areas add up
m, n = nevis.dimensions()
print(m * n // 2500)
print(int(np.sum(area)))
assert int(np.sum(area)) == m * n // 2500

364000000
364000000


In [3]:
# check that the i-th local max belongs to the i-th b.o.a.
len_maxima = len(maxima)
for i in range(len_maxima):
    assert i == label[maxima[i, 0], maxima[i, 1]]

In [4]:
# a two-line tour to ben nevis
x, y = maxima[0]
h[x-1:x+2, y-1:y+2]

array([[1343.1, 1342.1, 1340.9],
       [1339.6, 1345.3, 1343.8],
       [1222.5, 1299.1, 1327.7]], dtype=float32)

### Calculation of Table 1 & Table 2

In [5]:
intervals = [
    (1340, 1350),
    (1310, 1340),
    (1297, 1310),
    (1235, 1297),
    (1215, 1235),
    (1150, 1215),
    (1100, 1150),
    (1000, 1100),
    (600, 1000),
    (0, 600),
    (-100, 0)
]

In [6]:
from collections import defaultdict

# sum of area of b.o.a. of an interval
interval_area = defaultdict(int)
# sum of path length of b.o.a. of an interval
interval_sum_path = defaultdict(float)
# sum of all areas (check)
s_prime = 0

for i, (x, y) in enumerate(list(maxima)):
    z = h[x, y]

    # find the interval this height belongs to
    for a, b in intervals:
        if a <= z < b:
            break
    else:
        print(z, h[x, y])
    
    interval_area[a] += area[i]
    s_prime += area[i]
    interval_sum_path[a] += path_sum[i]

In [7]:
# check that the sum of areas add up
s_prime

364000000

This is the data for Table 1:

In [8]:
interval_area

defaultdict(int,
            {1340: 878,
             1310: 2181,
             1297: 1322,
             1235: 11789,
             1215: 4988,
             1150: 41411,
             1100: 66998,
             1000: 262135,
             600: 5766304,
             0: 357714202,
             -100: 127792})

In [9]:
interval_sum_path

defaultdict(float,
            {1340: 23124.0,
             1310: 73531.0,
             1297: 24059.0,
             1235: 242208.0,
             1215: 111350.0,
             1150: 936360.0,
             1100: 1273818.0,
             1000: 5910268.0,
             600: 116073494.0,
             0: 162857825980.0,
             -100: 496888.0})

In [10]:
# average function eval for a run which ends up in a local max in this interval
interval_avg_fe = {}
for k in interval_sum_path.keys():
    interval_avg_fe[k] = interval_sum_path[k] / interval_area[k]

interval_avg_fe

{1340: 26.337129840546698,
 1310: 33.714351215038974,
 1297: 18.198940998487142,
 1235: 20.545254050385953,
 1215: 22.323576583801124,
 1150: 22.611383448842094,
 1100: 19.01277650079107,
 1000: 22.546657256757015,
 600: 20.129617515829896,
 0: 455.27358172936056,
 -100: 3.8882559158632777}

In [11]:
s = s_prime
# the lower bound for each interval, ordered from low to high
u = [x for x, _ in intervals][::-1]

# convert the dictionary interval_avg_fe to a list
a = [interval_avg_fe[x] for x in u ]

# the probablity of a random initial point to start 
# in an b.o.a. whose local max belongs to each interval
p = [interval_area[x] / s for x in u]

In [12]:
u

[-100, 0, 600, 1000, 1100, 1150, 1215, 1235, 1297, 1310, 1340]

In [13]:
a

[3.8882559158632777,
 455.27358172936056,
 20.129617515829896,
 22.546657256757015,
 19.01277650079107,
 22.611383448842094,
 22.323576583801124,
 20.545254050385953,
 18.198940998487142,
 33.714351215038974,
 26.337129840546698]

In [14]:
p

[0.0003510769230769231,
 0.9827313241758242,
 0.015841494505494505,
 0.0007201510989010989,
 0.00018406043956043956,
 0.00011376648351648352,
 1.3703296703296703e-05,
 3.238736263736264e-05,
 3.6318681318681317e-06,
 5.9917582417582414e-06,
 2.4120879120879122e-06]

In [15]:
sum(p) # this should be one

1.0

In [16]:
def calc(p, a, k, n):
    return sum(p[i] * a[i] for i in range(n)) / (1 - sum(p[i] for i in range(k - 1)))
n = len(u) # number of intervals
for k in range(n):
    print(u[k], calc(p, a, k, n))
# this is the data for Table 2

-100 447.75546999999995
0 447.75546999999995
600 447.91272182018355
1000 26466.845124866722
1100 416089.2491741349
1150 1257905.107627923
1215 2604852.100561725
1235 7703137.871258329
1297 10079343.913415886
1310 37202234.895991854
1340 53279827.093737766


In [17]:
# convert the data to latex format
def to_latex(float_number):
    import math
    # Extracting the exponent part
    exponent = int(math.floor(math.log10(abs(float_number))))

    # Extracting the mantissa part
    mantissa = float_number / (10 ** exponent)

    # Formatting the float number in the LaTeX style format
    latex_formatted_number = "\\({:.2f} \\times 10^{{{}}}\\)".format(mantissa, exponent)
    return latex_formatted_number

In [18]:
n = len(u)
for k in range(n):
    print('\\(', u[k], '\\)', '&', to_latex(calc(p, a, k, n)), '\\\\')

\( -100 \) & \(4.48 \times 10^{2}\) \\
\( 0 \) & \(4.48 \times 10^{2}\) \\
\( 600 \) & \(4.48 \times 10^{2}\) \\
\( 1000 \) & \(2.65 \times 10^{4}\) \\
\( 1100 \) & \(4.16 \times 10^{5}\) \\
\( 1150 \) & \(1.26 \times 10^{6}\) \\
\( 1215 \) & \(2.60 \times 10^{6}\) \\
\( 1235 \) & \(7.70 \times 10^{6}\) \\
\( 1297 \) & \(1.01 \times 10^{7}\) \\
\( 1310 \) & \(3.72 \times 10^{7}\) \\
\( 1340 \) & \(5.33 \times 10^{7}\) \\


In [19]:
for a, c in interval_area.items():
    print(a, to_latex(c), '&', to_latex(c / s))

1340 \(8.78 \times 10^{2}\) & \(2.41 \times 10^{-6}\)
1310 \(2.18 \times 10^{3}\) & \(5.99 \times 10^{-6}\)
1297 \(1.32 \times 10^{3}\) & \(3.63 \times 10^{-6}\)
1235 \(1.18 \times 10^{4}\) & \(3.24 \times 10^{-5}\)
1215 \(4.99 \times 10^{3}\) & \(1.37 \times 10^{-5}\)
1150 \(4.14 \times 10^{4}\) & \(1.14 \times 10^{-4}\)
1100 \(6.70 \times 10^{4}\) & \(1.84 \times 10^{-4}\)
1000 \(2.62 \times 10^{5}\) & \(7.20 \times 10^{-4}\)
600 \(5.77 \times 10^{6}\) & \(1.58 \times 10^{-2}\)
0 \(3.58 \times 10^{8}\) & \(9.83 \times 10^{-1}\)
-100 \(1.28 \times 10^{5}\) & \(3.51 \times 10^{-4}\)


### Number of local maxima over 3000 feet

In [20]:
cnt_3000_feet = 0
cnt_all = 0

for x, y in maxima:
    z = h[x, y]
    if z >= 914.4:
        cnt_3000_feet += 1
    cnt_all += 1

print(cnt_3000_feet)
print(cnt_all)

1242
981133
